In [3]:
!pip install openml

    100% |████████████████████████████████| 143kB 5.2MB/s 
  Stored in directory: /home/rflperry/.cache/pip/wheels/ba/91/bb/74f93ae5dc60d713159def103484640861b338078fe978b8ee
  Stored in directory: /home/rflperry/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
Successfully built openml liac-arff


In [1]:
import gc
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import time
import math
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
from rerf.RerF import fastPredict, fastPredictPost, fastRerF
from rerf.rerfClassifier import rerfClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
from tqdm import tqdm
import seaborn as sns

import openml

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [16]:
#with open(os.path.expanduser('~/jovo-lab/RerF/Python/structured_experiments/openML_tests/.api_key.txt'), 'r') as f:
#    openml.config.apikey = f.readline()
    
#openml.config.cache_directory = os.path.expanduser('~/jovo-lab/RerF/Python/structured_experiments/openML_tests/cache/')

In [4]:
## Setup for run
num_runs = 1
#names = {"3-Nearest-Neighbors": "blue", "sklearn-RF":"#35B779FF",\
#        "ND-RerF-control":"purple", "ND-RF":"#f86000", "RerF": "black", "S-RerF":"#ff0000"}

names = {"S-RerF":"#ff0000",
         "sklearn-RF":"#35B779FF",
         "ND-RerF-control":"purple",
         "ND-RF":"#f86000", 
         "RerF": "black"}

n_est = 100
n_test = 10000
ns = [10,100,1000,5000]
ncores = 1
mx = 6
mn = 1

classifiers = [
    rerfClassifier(
        projection_matrix="S-RerF", 
        max_features = 28,
        n_estimators=n_est,
        n_jobs=ncores,
        image_height=28, 
        image_width=28, 
        patch_height_max=mx,
        patch_width_max=mn,
        patch_height_min=mx,
        patch_width_min=mn,
        ),
    RandomForestClassifier(n_estimators=n_est, max_features=28, n_jobs=ncores),
    rerfClassifier(n_estimators = n_est, projection_matrix = "RerF", 
        max_features = math.ceil(28*2.5*2.5), n_jobs = ncores),
    rerfClassifier(n_estimators = n_est, projection_matrix = "Base", 
        max_features = 28, n_jobs = ncores),
    rerfClassifier(n_estimators = n_est, projection_matrix = "RerF",
        max_features = 28, n_jobs = ncores),
    ]

In [52]:
# Get a task
task = openml.tasks.get_task(9983)

In [53]:
# Get data
data = openml.datasets.get_dataset(task.dataset_id)

In [54]:
data.description

"**Author**: Oliver Roesler  \n**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/EEG+Eye+State), Baden-Wuerttemberg, Cooperative State University (DHBW), Stuttgart, Germany  \n**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)  \n\nAll data is from one continuous EEG measurement with the Emotiv EEG Neuroheadset. The duration of the measurement was 117 seconds. The eye state was detected via a camera during the EEG measurement and added later manually to the file after analyzing the video frames. '1' indicates the eye-closed and '0' the eye-open state. All values are in chronological order with the first measured value at the top of the data.\n\nThe features correspond to 14 EEG measurements from the headset, originally labeled AF3, F7, F3, FC5, T7, P, O1, O2, P8, T8, FC6, F4, F8, AF4, in that order."

In [58]:
data.get_data()[0]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Class
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,1
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,1
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,1
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,1
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,1
5,4321.03,4004.62,4284.10,4153.33,4345.64,4587.18,4093.33,4616.92,4202.56,4232.82,4209.74,4281.03,4628.21,4389.74,1
6,4319.49,4001.03,4280.51,4151.79,4343.59,4584.62,4089.74,4615.90,4212.31,4226.67,4201.03,4269.74,4625.13,4378.46,1
7,4325.64,4006.67,4278.46,4143.08,4344.10,4583.08,4087.18,4614.87,4205.64,4230.26,4195.90,4266.67,4622.05,4380.51,1
8,4326.15,4010.77,4276.41,4139.49,4345.13,4584.10,4091.28,4608.21,4187.69,4229.74,4202.05,4273.85,4627.18,4389.74,1
9,4326.15,4011.28,4276.92,4142.05,4344.10,4582.56,4092.82,4608.72,4194.36,4228.72,4212.82,4277.95,4637.44,4393.33,1


In [7]:
clf = rerfClassifier(
        projection_matrix="S-RerF", 
        max_features = 28,
        n_estimators=n_est,
        n_jobs=ncores,
        image_height=28, 
        image_width=28, 
        patch_height_max=,
        patch_width_max=mn,
        patch_height_min=mx,
        patch_width_min=mn,
        )

rerfClassifier(feature_combinations=1.5, image_height=28, image_width=28,
               max_depth=None, max_features=28, min_parent=1, n_estimators=100,
               n_jobs=1, oob_score=False, patch_height_max=6,
               patch_height_min=6, patch_width_max=1, patch_width_min=1,
               projection_matrix='S-RerF', random_state=None)

In [5]:
# Build any classifier or pipeline
clf = classifiers[0]

# Run the flow
run = openml.runs.run_model_on_task(clf, task)

print(run)

AttributeError: 'NoneType' object has no attribute 'seed_model'